In [1]:
%cd ..

/workspace/shai_hulud/ppc_experiments


In [2]:
import argparse
import collections
import numpy as np
import pyro
import torch
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
import trainer.trainer as module_trainer
import utils

In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
np.random.seed(SEED)

In [6]:
from utils import read_json

config = read_json("experiments/ppc_celeba_config.json")
config = ConfigParser(config)

In [7]:
logger = config.get_logger('train')

# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

# build model architecture, then print to console
model = config.init_obj('arch', module_arch)
logger.info(model)

# get function handles of metrics
metrics = [getattr(module_metric, met) for met in config['metrics']]

# build optimizer.
optimizer = config.init_obj('optimizer', pyro.optim)

# build trainer
# kwargs = config['trainer'].pop('args')
trainer = config.init_obj('trainer', module_trainer, model, metrics, optimizer,
                          config=config, data_loader=data_loader,
                          valid_data_loader=valid_data_loader,
                          lr_scheduler=None)

trainer._resume_checkpoint("saved/models/CelebA_Ppc/0503_230620/checkpoint-epoch250.pth")

Files already downloaded and verified
CelebAPpc(
  (prior): GaussianPrior()
  (likelihood): ConvolutionalDecoder(
    (linear): Sequential(
      (0): Linear(in_features=40, out_features=256, bias=True)
      (1): SiLU()
    )
    (convs): Sequential(
      (0): ConvTranspose2d(256, 64, kernel_size=(4, 4), stride=(1, 1))
      (1): SiLU()
      (2): ConvTranspose2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): SiLU()
      (4): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (5): SiLU()
      (6): ConvTranspose2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (7): SiLU()
      (8): ConvTranspose2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (9): SiLU()
      (10): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    )
  )
  (graph): PpcGraphicalModel()
)
Trainable parameters: 454627
Initialize particles: train batch 0
Initialize particles: train batch 1
Initialize pa

In [8]:
trainer.model.eval()
trainer.cpu()
trainer.valid_particles.cpu()

ParticleDict(  (z): Parameter containing: [torch.FloatTensor of size 2x16277x40])

In [9]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.nodes[site]['is_observed'] = trainer.model.graph.nodes[site]['value'] is not None

In [ ]:
xs, _, indices = list(trainer.valid_data_loader)[0]
trainer._load_particles(indices, False)
with pyro.plate_stack("forward", (trainer.num_particles, len(xs))):
    model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
    x_hats = model().mean(dim=0)

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=8, sharex="all", sharey="all", layout="compressed")

for i in range(8):
    orgs = xs[i].squeeze().detach().transpose(0, -1)
    estimates = x_hats[i].squeeze().detach().transpose(0, -1)
    axes[0, i].imshow(orgs)
    axes[1, i].imshow(estimates)

fig.savefig("ppc_celeba_recons.pdf", bbox_inches="tight")
plt.show()

In [ ]:
del xs
del x_hats

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.unclamp(site)

In [ ]:
with pyro.plate_stack("forward", (trainer.num_particles, 8)):
    x_hats = trainer.model().mean(dim=0)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=8, sharex="all", sharey="all", layout="compressed")

for i in range(8):
    estimates = x_hats[i].squeeze().detach().transpose(0, -1)
    axes[i].imshow(estimates)

fig.savefig("ppc_celeba_samples.pdf", bbox_inches="tight")
plt.show()

In [ ]:
del x_hats

In [ ]:
SEEDS = [123, 456, 789, 101112, 131415]

In [ ]:
LOG_LIKELIHOODS = torch.zeros(len(SEEDS), requires_grad=False)
MEAN_SQUARED_ERROR = torch.zeros(len(SEEDS), requires_grad=False)

In [ ]:
for (s, SEED) in enumerate(SEEDS):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    for b, (xs, target, indices) in enumerate(trainer.valid_data_loader):
        trainer._load_particles(indices, False)
    
        with pyro.plate_stack("forward", (trainer.num_particles, len(xs))):
            model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
            trace = pyro.poutine.trace(model).get_trace()
            x_hats = trace.nodes['X']['value'].mean(dim=0)
        LOG_LIKELIHOODS[s] += trace.nodes['X']['fn'].log_prob(xs).sum()
        MEAN_SQUARED_ERROR[s] += ((xs - x_hats) ** 2).sum(dim=0).mean()

        del xs
        del x_hats
        logger.info("Evaluated likelihood for valid batch %d under seed %s" % (b, s))

    LOG_LIKELIHOODS[s] /= len(trainer.valid_data_loader.sampler)
    MEAN_SQUARED_ERROR[s] /= len(trainer.valid_data_loader.sampler)

In [ ]:
LOG_LIKELIHOODS.mean(), LOG_LIKELIHOODS.std()

In [ ]:
MEAN_SQUARED_ERROR.mean(), MEAN_SQUARED_ERROR.std()